In [38]:
import numpy as np
import pandas as pd

In [50]:
data = pd.read_csv('../dataset/transform/data100.csv')
data = data.fillna(0)
data.head()

,modulation,timeDomain_1,timeDomain_2,timeDomain_3,timeDomain_4,timeDomain_5,timeDomain_6,timeDomain_7,timeDomain_8,timeDomain_9,...,cs_prior__1021,cs_prior__1022,cs_prior__1023,cs_prior__1024,sDomain_max,sDomain_min,sDomain_mean,sDomain_1st_Qu,sDomain_2nd_Qu,sDomain_3rd_Qu
0,8PSK,0.639289,-0.213478,-0.865621,0.386868,0.621987,-0.567360,0.010030,0.373295,-0.150273,...,-0.939505,-0.937342,-1.158446,-1.151277,38.690961,0.037595,5.076279,0.955909,1.621237,7.163684
1,16QAM,-0.343101,-0.515628,-0.672854,0.547040,1.419239,-0.167272,-1.138051,-0.069713,1.464533,...,-1.620745,-1.181503,-1.134026,-0.515628,47.539106,0.000846,5.453533,1.893632,3.143240,5.845227
2,64QAM,-1.054600,-0.523152,0.886420,0.546905,-0.485738,-0.524877,-0.020109,0.228228,0.193880,...,-0.590973,-1.065564,-1.218445,-0.738838,38.200695,0.000383,4.535727,0.325565,0.591289,6.332244
3,BPSK,0.201801,0.990348,-0.083518,-0.947808,0.114776,0.990819,-0.046717,-1.288954,0.392954,...,-0.754663,-0.737908,-0.806254,-0.696160,37.163558,0.006632,4.472471,0.413386,0.768464,5.450173
4,CPFSK,0.168200,0.239790,-0.023122,-0.187393,-0.080768,0.208151,0.165164,-0.273555,-0.237533,...,-1.300745,-1.395009,-1.410663,-1.383565,59.125770,0.056253,4.774119,0.993590,1.838552,4.677182


In [51]:
y = data.modulation
X = data.drop(columns = "modulation").to_numpy()

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

algorithms = {
    "RF": (RandomForestClassifier(), { "max_depth": [100], "n_estimators": [10] }),
    "RF_Est" : (RandomForestClassifier(), { "criterion": ("entropy"), "max_depth": [100], "n_estimators": [10] })
}

In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer

gskf = StratifiedKFold(n_splits=10, shuffle=True, random_state=17) 
perf = accuracy_score

correct = {}
for algorithm in algorithms.keys():
    correct[algorithm] = y.copy()
    
score = {}
for algorithm in algorithms.keys():
    score[algorithm] = []
    
for algorithm, (clf, parameters) in algorithms.items():
    for train, test in kf.split(X, y):
        prep = StandardScaler()
        prep.fit(X[train])
        print(parameters)
        best = GridSearchCV(clf, parameters, cv=gskf, scoring="accuracy")
        best.fit(prep.transform(X[train]), y[train])
        
        predicted = best.predict(prep.transform(X[test]))
        
        correct[algorithm].loc[test] = predicted == y[test]
        score[algorithm].append(perf(best.predict(X[test]), y[test]))

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class

ValueError: Parameter grid for parameter (criterion) needs to be a list or numpy array, but got (<class 'str'>). Single values need to be wrapped in a list with one element.

In [ ]:
import numpy as np
from mlxtend.evaluate import mcnemar_table

In [ ]:
# The correct target (class) labels
idx_modulation = {"8PSK":0, "16QAM":1, "64QAM":2, "BPSK":3, "CPFSK":4, "GFSK":5, "QPSK":6, "PAM4":7}
y_target = y.replace(idx_modulation)

# Class labels predicted by model RandomForestClassifier
y_RF = pd.DataFrame.from_dict(correct).RF

# Class labels predicted by model DecisionTreeClassifier
y_DT = pd.DataFrame.from_dict(correct)['RF-Est.'] 

tb = mcnemar_table(y_target=y_target, 
                   y_model1=y_RF, 
                   y_model2=y_DT)

ct=pd.DataFrame(tb, columns=["RF positive", "RF negative"])
ct.index=["RF-Est. positive", "RF-Est. negative"] 
ct

In [ ]:
from sklearn.preprocessing import StandardScaler
prep = StandardScaler()
prep.fit(iris.data)
clf.predict(prep.transform(iris.data))
clf.score(iris.data)